## Landau Damping in VlaPy
### Archis Joglekar
### August 16, 2020


This notebook provides an example of how one can use VlaPy. Here, we verify that the code reproduces Landau Damping for $k=0.3$.  To initialize the example, we apply an external force to the system for a short period of time. After this time, we observe evidence of damping of the electric field at the rate specified by Landau Damping theory.

### Import Packages

In [1]:
import numpy as np

from vlapy import manager, initializers
from vlapy.infrastructure import print_to_screen, mlflow_helpers
from vlapy.diagnostics import landau_damping, z_function
import uuid

## Perform 1 run

### Initialize simulation parameters and driver to test Landau Damping for a random wavenumber
### This is performed in 2 steps...
### 1 - Get Default Simulation Parameters
### 2 - Initialize Driver According to the EPW resonance condition

In [2]:
k0 = np.random.uniform(0.3, 0.4)

In [3]:
# Initialize all default parameters
all_params_dict = initializers.make_default_params_dictionary()

# Acquire the EPW resonance conditions and initialize the spatial domain accordingly
all_params_dict = initializers.specify_epw_params_to_dict(
    k0=k0, all_params_dict=all_params_dict
)

# Collisionless Simulation
all_params_dict["nu"] = 0. 


# Here we create the pulse dictionary
pulse_dictionary = {
    "first pulse": {
        "start_time": 0,
        "rise_time": 5,
        "flat_time": 10,
        "fall_time": 5,
        "w0": all_params_dict["w_epw"],
        "a0": all_params_dict["a0"],
        "k0": k0,
    }
}

mlflow_exp_name = "vlapy-test"

uris = {
    "tracking": "local",
}

print_to_screen.print_startup_message(
    mlflow_exp_name, all_params_dict, pulse_dictionary
)

Starting VlaPy at 08/16/2020 13:46:51
MLFlow experiment name: vlapy-test
Run parameters: 
{'a0': 1e-07,
 'backend': {'core': 'numpy', 'max_doubles_per_file': 100000000},
 'fokker-planck': {'solver': 'batched_tridiagonal', 'type': 'lb'},
 'nt': 1000,
 'nu': 0.0,
 'nu_ld': -0.05779098807257614,
 'nv': 1024,
 'nx': 64,
 'tmax': 100,
 'v_ph': 3.2710604304418864,
 'vlasov-poisson': {'edfdv': 'exponential',
                    'poisson': 'spectral',
                    'time': 'leapfrog',
                    'vdfdx': 'exponential'},
 'vmax': 6.4,
 'w_epw': 1.2697545343602237,
 'xmax': 16.18634017779242,
 'xmin': 0.0}
Driver parameters: 
{'first pulse': {'a0': 1e-07,
                 'fall_time': 5,
                 'flat_time': 10,
                 'k0': 0.38817825636706227,
                 'rise_time': 5,
                 'start_time': 0,
                 'w0': 1.2697545343602237}}



### The simulation parameters are listed above. Assuming those check out, the simulation continues below. After it ends, we print the measured Landau Damping rate and compare it with the Actual

In [4]:
that_run = manager.start_run(
    all_params=all_params_dict,
    pulse_dictionary=pulse_dictionary,
    diagnostics=landau_damping.LandauDamping(
        vph=all_params_dict["v_ph"], wepw=all_params_dict["w_epw"],
    ),
    uris=uris,
    name=mlflow_exp_name,
)

print (
    "Measured Landau Damping Rate: " + str(mlflow_helpers.get_this_metric_of_this_run("damping_rate", that_run)),
    "Actual Landau Damping Rate: " + str(all_params_dict["nu_ld"]),
)

100%|██████████| 1/1 [00:24<00:00, 24.91s/it]

Measured Landau Damping Rate: -0.05769441438284097 Actual Landau Damping Rate: -0.05779098807257614


## Note that the damping rate is as expected within a small error.

## The data and metadata from that simulation can be found by launching the MLFlow UI
This requires one to launch an MLFlow server by executing `mlflow ui` from the directory of execution (`VlaPy/notebooks`).
An MLFlow UI becomes accessible at `http://localhost:5000` . We provide an example screenshot:

![UI example](screenshots_for_example/ui.png)

## This UI records any parameters and metrics. 


![params](screenshots_for_example/params_metrics.png)

## Most importantly, it also shows where the files were stored for that particular run.

![Plot example](screenshots_for_example/damping.png)